In [ ]:
import genjax
import jax.numpy as jnp
import jax.random as jr
from genjax import ChoiceMapBuilder as C
from genjax import pretty

pretty()

foo = genjax.gen(lambda loc: genjax.normal(loc, 1.0) @ "x")

In [ ]:
foo.simulate(jr.PRNGKey(0), (0.0,))

In [ ]:
tr = foo.vmap(in_axes=(0,)).simulate(jr.PRNGKey(0), (jnp.zeros(3),))
tr

In [ ]:
@genjax.gen
def regular():
    x = genjax.normal(0.0, 1.0) @ "x"
    return x


@genjax.gen
def outlier():
    x = genjax.normal(0.0, 10.0) @ "x"
    return x


model = genjax.switch(regular, outlier)
tr = model.simulate(jr.PRNGKey(0), (0, (), ()))

update = genjax.GenericProblem(
    (
        genjax.Diff((0), genjax.UnknownChange),
        genjax.Diff((), genjax.NoChange),
        genjax.Diff((), genjax.NoChange),
    ),
    C.n(),
)

model.update(jr.PRNGKey(1), tr, update)

# @genjax.gen
# def q(ix):
#     branches = [regular, outlier]
#     args = [(1,), (1,)]
#     v = genjax.switch(*branches)(ix, *args) @ "switch"
#     return v
# tr = q.simulate(jr.PRNGKey(0), (1,))

# tr.update(jr.PRNGKey(1), genjax.EmptyChoiceMap(), (genjax.diff(1, genjax.UnknownChange),))

In [ ]:
import genjax
import jax.numpy as jnp
from genjax import ChoiceMapBuilder as C
from genjax import gen
from jax import jit, random

key = random.PRNGKey(0)
N = 4
F = 4
chm = C[
    jnp.arange(4), jnp.repeat(jnp.arange(N)[jnp.newaxis], F, axis=0), "new_pixel"
].set(3.0)

image = jnp.zeros([4, 4], dtype=jnp.float32)
sample_image = genjax.vmap(in_axes=(0,))(
    genjax.vmap(in_axes=(0,))(
        gen(lambda pixel: genjax.normal(pixel, 1.0) @ "new_pixel")
    )
)

tr, w = jit(sample_image.importance)(key, chm, (image,))